In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
df = pd.read_csv("data.csv", sep=",")
print(df.columns)
df = df.drop(["instant", "dteday", "casual", "registered"], axis=1)

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')


In [5]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = StandardScaler()

In [6]:
X = df.values[:,:-1]
X = scaler.fit_transform(X)
Y = df.values[:,-1]
Y = Y.reshape(-1,1)


df = pd.DataFrame(np.concatenate([X,Y], axis=1), columns=df.columns)

print(df.head)



<bound method NDFrame.head of          season        yr      mnth        hr  holiday   weekday  workingday  \
0     -1.349864 -1.003866 -1.603121 -1.668944 -0.17149  1.494950   -1.460672   
1     -1.349864 -1.003866 -1.603121 -1.524341 -0.17149  1.494950   -1.460672   
2     -1.349864 -1.003866 -1.603121 -1.379739 -0.17149  1.494950   -1.460672   
3     -1.349864 -1.003866 -1.603121 -1.235137 -0.17149  1.494950   -1.460672   
4     -1.349864 -1.003866 -1.603121 -1.090534 -0.17149  1.494950   -1.460672   
...         ...       ...       ...       ...      ...       ...         ...   
10881  1.338012  0.996149  1.590639  1.078500 -0.17149  0.000686    0.684616   
10882  1.338012  0.996149  1.590639  1.223102 -0.17149  0.000686    0.684616   
10883  1.338012  0.996149  1.590639  1.367704 -0.17149  0.000686    0.684616   
10884  1.338012  0.996149  1.590639  1.512307 -0.17149  0.000686    0.684616   
10885  1.338012  0.996149  1.590639  1.656909 -0.17149  0.000686    0.684616   

       we

In [7]:
train = df.sample(frac=0.8,random_state=210)
test =df.drop(train.index)                        

In [8]:
import torch.utils.data as data
train_dataset = data.TensorDataset(torch.tensor(train.values[:,:-1], dtype=torch.float32),
                                   torch.tensor(train.values[:,-1], dtype=torch.float32).unsqueeze(1))
test_dataset = data.TensorDataset(torch.tensor(test.values[:,:-1], dtype=torch.float32),
                                   torch.tensor(test.values[:,-1], dtype=torch.float32).unsqueeze(1))

print(train.values[:,-1])

train_dataloader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = data.DataLoader(test_dataset, batch_size=32, shuffle=True)

[353. 315. 171. ... 217. 305. 147.]


In [9]:
import torch.nn as nn
import torch.nn.functional as F


class MyRegressor(nn.Module):
    def __init__(self, num_in, num_h, num_out):
        super().__init__()
        self.linear1 = nn.Linear(num_in, num_h)
        self.act_fn = nn.LeakyReLU()
        self.linear2 = nn.Linear(num_h,num_out)

    def forward(self,x):
        x = self.linear1(x)
        x = self.act_fn(x)
        x = self.linear2(x)
        return x

In [16]:
device = torch.device("cuda")
model = MyRegressor(12, 30, 1)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_module = nn.MSELoss()

In [17]:
model.train()

for i in range(5000):
    for X,Y in train_dataloader:
        X = X.to(device)
        Y = Y.to(device)
        
        preds = model(X)
        loss = loss_module(preds,Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {i}, loss: {loss.item():.3}")


Epoch 0, loss: 6.87e+04
Epoch 1, loss: 4.64e+04
Epoch 2, loss: 1.19e+05
Epoch 3, loss: 4.3e+04
Epoch 4, loss: 2.04e+04
Epoch 5, loss: 1.64e+03
Epoch 6, loss: 7.1e+04
Epoch 7, loss: 6.54e+03
Epoch 8, loss: 7.97e+04
Epoch 9, loss: 3.46e+03
Epoch 10, loss: 6.97e+04
Epoch 11, loss: 9.59e+03
Epoch 12, loss: 2.43e+03
Epoch 13, loss: 1.82e+03
Epoch 14, loss: 3.4e+04
Epoch 15, loss: 2.71e+04
Epoch 16, loss: 7.97e+03
Epoch 17, loss: 3.36e+04
Epoch 18, loss: 2.52e+04
Epoch 19, loss: 1.07e+04
Epoch 20, loss: 5.4e+04
Epoch 21, loss: 1.32e+04
Epoch 22, loss: 4.68e+04
Epoch 23, loss: 8.56e+03
Epoch 24, loss: 3.76e+04
Epoch 25, loss: 7.83e+02
Epoch 26, loss: 1.76e+04
Epoch 27, loss: 2.78e+04
Epoch 28, loss: 1.38e+04
Epoch 29, loss: 1.24e+04
Epoch 30, loss: 4.82e+03
Epoch 31, loss: 7.09e+03
Epoch 32, loss: 1.12e+04
Epoch 33, loss: 2.79e+04
Epoch 34, loss: 5.5e+03
Epoch 35, loss: 1.53e+04
Epoch 36, loss: 5.74e+03
Epoch 37, loss: 1.14e+04
Epoch 38, loss: 1.18e+04
Epoch 39, loss: 6.25e+03
Epoch 40, loss:

In [18]:
model.eval()

predictions = []
targets = []

with torch.no_grad():
    for X, Y in test_dataloader:
        X = X.to(device)
        Y = Y.to(device)
        preds = model(X)
        targets.append(Y)
        predictions.append(preds)


def rmsle(y_true,y_pred):
    n = len(y_true)
    msle = np.mean([(np.log(max(y_pred[i],0) + 1) - np.log(y_true[i] + 1)) ** 2.0 for i in range(n)])
    return np.sqrt(msle)
targets = torch.cat(targets, dim =0 ).cpu().numpy()
predictions = torch.cat(predictions, dim=0).cpu().numpy()

print(rmsle(targets, predictions))

1.0032266077860084
